## Import Libraries

In [44]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Olympics Games Year and Location

In [45]:
import pandas as pd

# Read the CSV file into a DataFrame
df_Games = pd.read_csv("Olympics_Games.csv")

# Filter for rows where the edition is "Summer"
df_Summer_Games = df_Games[df_Games['edition'].str.contains("Summer")].sort_values(by='year')

# Select relevant columns
df_Summer_Games = df_Summer_Games[['edition_id', 'edition', 'city', 'year', 'competition_date']]

# Replace missing values in 'competition_date' for specific 'edition_id' values
df_Summer_Games.loc[df_Summer_Games['edition_id'].isin([50, 51, 52]), 'competition_date'] = "Not Held Due to War"

# Display the DataFrame
df_Summer_Games.head()


,edition_id,edition,city,year,competition_date
0,1,1896 Summer Olympics,Athina,1896,6 – 13 April
1,2,1900 Summer Olympics,Paris,1900,14 May – 28 October
2,3,1904 Summer Olympics,St. Louis,1904,1 July – 26 November
3,5,1908 Summer Olympics,London,1908,27 April – 31 October
4,6,1912 Summer Olympics,Stockholm,1912,5 May – 27 July


## Olympics Medal Tally by Countries Overall

In [54]:
df_Medal = pd.read_csv("Olympic_Games_Medal_Tally.csv")

# Filter for rows where the edition is 'Summer'
df_Medal = df_Medal[df_Medal['edition'].str.contains("Summer")]

# Group by country and sum the medal counts
df_Medal = df_Medal.groupby('country').sum()

# Select relevant columns to display
df_Medal = df_Medal[['gold', 'silver', 'bronze', 'total']]

# Sort by total medals in descending order
df_Medal = df_Medal.sort_values(by='total', ascending=False)

# Add rank column
df_Medal['Rank'] = df_Medal['total'].rank(ascending=False, method='first').astype(int)

# Reorder columns
df_Medal = df_Medal[['Rank','gold', 'silver', 'bronze', 'total']].reset_index()

df_Medal.head()

,country,Rank,gold,silver,bronze,total
0,United States,1,1070,841,744,2655
1,Soviet Union,2,395,319,296,1010
2,Great Britain,3,291,323,313,927
3,Germany,4,237,264,290,791
4,France,5,231,257,285,773
